# Carga de Librerias

In [82]:
## =========================================================================
## @author Andres Gonzalo Vargas Duran (andresg.vargasd@javeriana.edu.co)
## =========================================================================
import numpy, sys, time
import pandas
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
sys.path.append('C:\\Users\\andre\\Downloads\\PUJ_ML_03\\PUJ_ML_03\\lib\\python3\\')
import PUJ
import PUJ.Model.Logistic 
import joblib
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten,Conv2D
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Obtención y organización de los datos

In [27]:
### Lectura de datos

datos = pandas.read_csv('data/age_gender.csv')
datos['pixels']=datos['pixels'].apply(lambda x:  numpy.array(x.split(), dtype="float32"))
pixels = numpy.array(datos['pixels'].tolist())
age = numpy.ravel(datos['age'].tolist())
gender = numpy.array(datos['gender'].tolist())
ethnicity = numpy.array(datos['ethnicity'].tolist())

### Transposición

rows = len(age)
ageT = age
genderT = gender.reshape(rows,1)
ethnicityT = ethnicity.reshape(rows,1)

### Creación de datasets para cada variable

In [28]:
### Variable Género

x_trainG, x_testG, y_trainG, y_testG = train_test_split(pixels, genderT)
print('X_trainG: ' + str(x_trainG.shape))
print('Y_trainG: ' + str(y_trainG.shape))
print('X_testG:  '  + str(x_testG.shape))
print('Y_testG:  '  + str(y_testG.shape))

### Variable Etnia

x_trainE, x_testE, y_trainE, y_testE = train_test_split(pixels, ethnicityT)
print('X_trainE: ' + str(x_trainE.shape))
print('Y_trainE: ' + str(y_trainE.shape))
print('X_testE:  '  + str(x_testE.shape))
print('Y_testE:  '  + str(y_testE.shape))

### Variable Edad

x_trainA, x_testA, y_trainA, y_testA = train_test_split(pixels, ageT)
print('X_trainA: ' + str(x_trainA.shape))
print('Y_trainA: ' + str(y_trainA.shape))
print('X_testA:  '  + str(x_testA.shape))
print('Y_testA:  '  + str(y_testA.shape))

X_trainG: (17778, 2304)
Y_trainG: (17778, 1)
X_testG:  (5927, 2304)
Y_testG:  (5927, 1)
X_trainE: (17778, 2304)
Y_trainE: (17778, 1)
X_testE:  (5927, 2304)
Y_testE:  (5927, 1)
X_trainA: (17778, 2304)
Y_trainA: (17778,)
X_testA:  (5927, 2304)
Y_testA:  (5927,)


# Modelo para el género

#### Utilizando PUJ

In [67]:
gen = numpy.concatenate((x_trainG[:,:], y_trainG[:,:]), axis = 1)
PosG = gen[gen[ :, -1] == 1]
NegG = gen[gen[ :, -1] == 0]

print(len(PosG))
print(len(NegG))

numPosG = 2000
numNegG = 2000
numpy.random.shuffle( PosG )
numpy.random.shuffle( NegG )
datosG = numpy.concatenate( ( PosG[ : numPosG , : ], NegG[ : numNegG , : ] ), axis = 0 )
numpy.random.shuffle( datosG )

# Configure model
g0 = PUJ.Model.Logistic( )
g0.setParameters( [ 0 for i in range( datosG.shape[ 1 ] ) ] )
# print( 'Initial model = ' + str( g0 ) )

# Configure cost
J0 = PUJ.Model.Logistic.Cost( g0, datosG[ : , 0 : -1 ], datosG[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J0 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 10000 )
opt.setNumberOfDebugIterations( 200 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations :', opt.realIterations( ) )
print( '= Fitted model :', g0 )
print( '===========================================' )

y_real = datosG[ : , -1 : ]
y_est = g0.threshold( datosG[ : , 0 : -1 ] )
K0 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  K0[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1
# end for

print( K0 )

accuracy0 = (K0[0,0] + K0[1,1])/(K0[0,0] + K0[0,1] + K0[1,0] + K0[1,1])
print(accuracy0)

print(numpy.shape(g0))

end0 = time.time()



8462
9316
0 0.6897588583961526 0.003388322161792634
200 0.5738888178147138 0.0002031075833878182
400 0.5412396445655867 0.00013408072587972608
600 0.5181221045526169 0.00010034892931742867
800 0.5002986072392641 7.941655792487179e-05
1000 0.48594666909841444 6.498262601517535e-05
1200 0.4740616679486824 5.4432091915024294e-05
1400 0.4640147062204588 4.642486038880511e-05
1600 0.4553819289103558 4.018050116988414e-05
1800 0.4478637976759011 3.520615497043522e-05
2000 0.44124142061272503 3.1173312058097036e-05
2200 0.4353506896598259 2.785447041298017e-05
2400 0.4300659823067961 2.508744971585486e-05
2600 0.4252894266151077 2.2753916984086242e-05
2800 0.4209435932020491 2.0765829575997152e-05
3000 0.41696638902644134 1.905656291079083e-05
3200 0.4133074120356833 1.7574938103726012e-05
3400 0.40992530026262125 1.628110788060022e-05
3600 0.4067857724911996 1.514366645405385e-05
3800 0.40386015880017295 1.4137584051332475e-05
4000 0.4011242838182983 1.3242707932070719e-05
4200 0.39855760768

In [68]:
joblib.dump(g0, 'genero.joblib')

['genero.joblib']

In [69]:
modelogen = joblib.load('genero.joblib')

In [70]:
modelogen.threshold([192, 192, 197, 207, 213, 213, 206, 197, 194, 200, 206, 210, 207, 207, 209, 214, 217, 215, 211, 215, 215, 216, 215, 212, 210, 207, 205, 211, 201, 201, 202, 203, 198, 194, 189, 184, 183, 181, 173, 164, 164, 164, 176, 187, 193, 196, 193, 182, 176, 185, 188, 208, 213, 207, 202, 195, 192, 198, 206, 210, 208, 207, 208, 214, 219, 215, 211, 213, 213, 216, 215, 212, 210, 207, 205, 211, 201, 201, 202, 202, 196, 193, 192, 191, 184, 178, 169, 163, 166, 171, 178, 186, 189, 185, 185, 173, 179, 169, 184, 201, 206, 202, 203, 193, 194, 204, 206, 208, 210, 209, 209, 211, 212, 210, 210, 211, 211, 214, 215, 214, 213, 208, 208, 209, 201, 199, 201, 200, 196, 194, 194, 195, 186, 173, 166, 164, 166, 169, 174, 188, 191, 192, 185, 179, 177, 160, 178, 191, 186, 190, 197, 194, 195, 204, 206, 207, 208, 210, 210, 211, 212, 212, 212, 215, 215, 215, 215, 213, 212, 211, 209, 206, 205, 201, 200, 199, 198, 196, 194, 188, 184, 181, 172, 161, 154, 163, 171, 182, 192, 187, 191, 186, 141, 163, 178, 187, 183, 179, 188, 188, 189, 202, 204, 208, 208, 213, 215, 214, 217, 215, 215, 217, 217, 214, 213, 212, 211, 211, 211, 207, 206, 201, 200, 198, 197, 194, 192, 187, 183, 183, 171, 157, 149, 152, 163, 174, 181, 193, 181, 190, 128, 174, 175, 175, 173, 169, 178, 181, 184, 197, 205, 205, 207, 207, 208, 207, 210, 210, 210, 211, 211, 213, 211, 211, 209, 211, 211, 207, 206, 204, 200, 195, 192, 191, 190, 186, 182, 181, 166, 151, 148, 157, 157, 166, 183, 179, 181, 187, 71, 164, 164, 166, 164, 166, 174, 177, 182, 200, 206, 209, 206, 208, 200, 200, 207, 214, 213, 210, 208, 208, 209, 211, 207, 206, 205, 207, 203, 201, 200, 193, 186, 190, 189, 183, 180, 183, 161, 148, 149, 153, 146, 149, 165, 172, 174, 170, 94, 158, 173, 170, 167, 167, 171, 178, 183, 197, 201, 205, 211, 212, 218, 209, 207, 204, 206, 206, 209, 202, 204, 206, 206, 207, 199, 199, 198, 191, 195, 194, 187, 188, 189, 177, 179, 174, 166, 148, 147, 142, 140, 144, 138, 173, 163, 165, 82, 155, 183, 174, 175, 172, 170, 176, 183, 185, 177, 174, 173, 175, 167, 175, 188, 198, 197, 192, 194, 192, 200, 197, 195, 195, 194, 180, 191, 194, 187, 180, 177, 181, 167, 162, 178, 170, 155, 155, 149, 147, 136, 141, 139, 159, 150, 168, 84, 159, 176, 169, 170, 182, 178, 177, 168, 138, 132, 134, 142, 104, 98, 82, 75, 122, 146, 169, 185, 189, 194, 189, 187, 178, 173, 161, 156, 123, 106, 94, 98, 111, 117, 134, 142, 159, 162, 150, 149, 148, 133, 137, 141, 146, 159, 153, 99, 167, 169, 164, 177, 177, 176, 176, 158, 140, 150, 141, 157, 150, 128, 93, 77, 78, 111, 121, 165, 178, 185, 190, 181, 165, 163, 141, 109, 76, 53, 45, 56, 79, 107, 125, 115, 115, 137, 147, 152, 140, 137, 140, 138, 142, 157, 161, 219, 183, 156, 166, 175, 177, 183, 182, 169, 149, 161, 165, 131, 102, 86, 82, 69, 65, 109, 123, 138, 172, 182, 192, 174, 152, 134, 98, 76, 57, 44, 57, 85, 119, 133, 135, 105, 102, 116, 146, 147, 136, 146, 136, 125, 133, 159, 169, 202, 186, 150, 154, 171, 180, 189, 181, 175, 169, 161, 62, 65, 68, 74, 56, 44, 70, 81, 108, 154, 164, 190, 194, 169, 116, 88, 74, 66, 85, 93, 90, 73, 73, 85, 107, 140, 121, 118, 132, 153, 153, 143, 132, 127, 125, 144, 160, 162, 174, 148, 147, 176, 172, 187, 180, 171, 135, 114, 108, 115, 100, 53, 46, 70, 121, 95, 112, 134, 161, 199, 201, 183, 122, 82, 75, 55, 77, 117, 77, 31, 56, 57, 49, 95, 153, 147, 156, 157, 160, 150, 136, 144, 122, 138, 155, 160, 171, 131, 146, 169, 181, 189, 182, 169, 151, 137, 141, 147, 142, 145, 138, 130, 103, 103, 137, 126, 161, 201, 205, 192, 131, 91, 93, 60, 74, 147, 69, 38, 62, 108, 91, 103, 140, 164, 174, 148, 163, 151, 144, 145, 128, 139, 147, 164, 165, 146, 167, 174, 181, 193, 197, 186, 197, 178, 170, 156, 185, 163, 141, 138, 124, 137, 138, 134, 175, 202, 204, 193, 141, 93, 114, 84, 96, 128, 144, 143, 152, 136, 116, 135, 152, 158, 165, 169, 166, 163, 144, 137, 125, 138, 151, 179, 168, 177, 167, 167, 177, 196, 202, 200, 210, 204, 200, 181, 180, 163, 149, 138, 134, 118, 132, 164, 182, 203, 206, 195, 161, 103, 83, 70, 75, 118, 166, 164, 169, 170, 167, 176, 177, 173, 173, 167, 161, 164, 153, 134, 121, 147, 153, 171, 182, 173, 145, 169, 178, 188, 201, 211, 199, 209, 209, 210, 198, 190, 174, 170, 160, 160, 170, 168, 177, 199, 205, 195, 154, 128, 115, 96, 101, 117, 140, 160, 174, 184, 186, 188, 186, 177, 168, 164, 161, 156, 158, 131, 118, 144, 150, 139, 194, 188, 174, 160, 178, 187, 194, 206, 205, 205, 207, 209, 203, 197, 193, 186, 183, 179, 171, 182, 173, 195, 211, 189, 142, 131, 127, 136, 142, 146, 168, 183, 193, 204, 207, 201, 191, 183, 170, 171, 158, 156, 145, 131, 124, 130, 152, 105, 201, 171, 192, 173, 172, 187, 191, 203, 212, 212, 210, 209, 206, 206, 209, 202, 200, 176, 174, 177, 170, 192, 204, 183, 146, 137, 135, 141, 157, 166, 178, 194, 205, 209, 201, 198, 189, 187, 172, 174, 156, 155, 130, 147, 148, 102, 128, 98, 201, 158, 194, 173, 167, 184, 189, 190, 196, 207, 206, 205, 202, 204, 206, 200, 186, 162, 175, 174, 178, 196, 203, 188, 164, 147, 141, 132, 146, 162, 178, 193, 199, 204, 195, 194, 188, 180, 166, 168, 152, 151, 130, 141, 154, 133, 150, 96, 200, 190, 174, 174, 166, 184, 186, 187, 193, 209, 211, 212, 208, 208, 206, 202, 180, 177, 177, 181, 178, 199, 203, 189, 170, 149, 144, 138, 152, 155, 166, 188, 194, 195, 190, 189, 188, 171, 161, 154, 152, 143, 134, 144, 162, 165, 145, 90, 204, 209, 173, 175, 163, 181, 189, 192, 191, 201, 211, 214, 208, 203, 198, 200, 190, 162, 178, 162, 174, 198, 198, 193, 176, 144, 139, 122, 139, 165, 173, 177, 183, 189, 197, 192, 185, 164, 155, 141, 152, 139, 136, 145, 169, 172, 131, 65, 194, 204, 197, 182, 154, 178, 195, 190, 193, 202, 207, 209, 207, 203, 198, 194, 177, 162, 180, 169, 187, 207, 204, 196, 191, 162, 146, 142, 121, 149, 165, 167, 176, 189, 195, 189, 174, 164, 151, 147, 142, 138, 141, 144, 163, 147, 158, 61, 170, 192, 186, 183, 154, 172, 187, 192, 193, 195, 205, 206, 204, 200, 196, 190, 162, 173, 185, 172, 194, 202, 197, 199, 195, 167, 144, 136, 125, 132, 137, 162, 173, 188, 190, 183, 170, 160, 151, 149, 140, 142, 142, 145, 159, 152, 188, 71, 127, 201, 194, 197, 163, 159, 186, 189, 190, 198, 204, 205, 204, 199, 194, 187, 158, 147, 120, 150, 180, 189, 191, 178, 175, 139, 100, 109, 126, 142, 129, 149, 169, 185, 183, 179, 162, 158, 151, 146, 141, 146, 139, 140, 165, 176, 183, 83, 70, 154, 200, 187, 162, 161, 174, 178, 187, 196, 207, 209, 205, 200, 194, 185, 171, 138, 121, 127, 148, 164, 165, 149, 132, 98, 80, 97, 129, 139, 140, 149, 170, 183, 177, 174, 156, 152, 152, 144, 145, 149, 139, 147, 155, 163, 172, 70, 67, 60, 112, 134, 165, 162, 163, 180, 185, 194, 200, 201, 201, 198, 192, 189, 171, 158, 147, 149, 144, 140, 131, 100, 93, 98, 118, 132, 136, 143, 154, 163, 179, 189, 182, 171, 154, 145, 152, 148, 141, 156, 140, 167, 167, 175, 158, 75, 58, 53, 41, 37, 137, 170, 167, 181, 183, 190, 194, 195, 196, 195, 196, 185, 177, 181, 174, 175, 154, 153, 151, 153, 145, 151, 159, 159, 155, 156, 154, 178, 185, 183, 183, 169, 152, 146, 153, 146, 149, 154, 136, 161, 167, 140, 57, 72, 68, 59, 54, 49, 115, 165, 174, 179, 185, 187, 189, 192, 190, 192, 194, 182, 190, 186, 183, 185, 163, 161, 157, 173, 161, 160, 159, 160, 157, 150, 150, 171, 177, 177, 173, 168, 158, 150, 151, 143, 155, 142, 43, 67, 40, 11, 29, 78, 77, 68, 59, 55, 106, 162, 173, 177, 187, 186, 187, 194, 187, 189, 183, 181, 186, 182, 180, 191, 180, 170, 168, 173, 172, 168, 162, 165, 157, 150, 153, 164, 177, 181, 168, 166, 159, 152, 151, 144, 154, 139, 6, 7, 14, 10, 18, 84, 83, 76, 65, 55, 97, 161, 170, 177, 188, 185, 186, 185, 180, 185, 166, 169, 179, 186, 185, 191, 181, 177, 176, 177, 174, 168, 162, 155, 136, 130, 142, 162, 165, 183, 170, 172, 154, 152, 149, 150, 153, 122, 4, 8, 13, 17, 15, 83, 82, 74, 66, 58, 88, 161, 170, 179, 187, 185, 187, 185, 189, 175, 150, 159, 169, 174, 177, 173, 171, 178, 184, 176, 145, 124, 125, 123, 117, 103, 101, 158, 172, 175, 172, 162, 155, 152, 149, 147, 153, 96, 8, 15, 22, 20, 18, 82, 78, 72, 65, 61, 80, 166, 162, 173, 185, 183, 186, 193, 186, 164, 125, 115, 94, 94, 95, 87, 63, 85, 102, 75, 47, 67, 68, 63, 63, 95, 99, 146, 181, 177, 169, 152, 156, 151, 146, 144, 152, 60, 20, 23, 22, 21, 21, 79, 75, 71, 67, 62, 66, 138, 158, 167, 183, 181, 186, 196, 182, 171, 173, 172, 173, 156, 171, 159, 164, 166, 161, 153, 149, 155, 152, 143, 139, 133, 141, 158, 190, 189, 170, 145, 160, 148, 143, 148, 149, 42, 30, 22, 16, 16, 16, 75, 72, 70, 67, 62, 78, 152, 166, 163, 178, 182, 176, 189, 189, 180, 185, 170, 164, 170, 172, 167, 172, 162, 150, 144, 154, 156, 152, 135, 123, 141, 153, 164, 180, 195, 162, 146, 147, 138, 143, 143, 136, 76, 24, 16, 9, 13, 12, 70, 69, 70, 67, 59, 65, 163, 154, 163, 165, 176, 172, 179, 189, 186, 189, 182, 174, 165, 168, 166, 154, 148, 142, 140, 147, 143, 137, 135, 135, 150, 163, 164, 172, 179, 148, 144, 144, 135, 142, 141, 136, 96, 41, 18, 13, 16, 13, 70, 67, 63, 63, 54, 67, 177, 150, 163, 151, 171, 169, 173, 185, 187, 191, 186, 177, 180, 165, 162, 156, 160, 161, 144, 155, 160, 157, 155, 158, 154, 164, 164, 160, 162, 133, 141, 144, 134, 135, 126, 159, 96, 47, 22, 15, 17, 15, 71, 66, 62, 60, 60, 57, 183, 166, 146, 173, 168, 166, 169, 180, 185, 187, 191, 190, 188, 187, 182, 178, 172, 171, 171, 168, 169, 171, 176, 166, 157, 167, 164, 156, 150, 126, 141, 143, 132, 129, 134, 130, 78, 37, 20, 12, 9, 11, 70, 65, 61, 58, 58, 61, 181, 175, 150, 161, 169, 166, 165, 177, 187, 190, 193, 196, 194, 192, 189, 186, 184, 180, 175, 172, 173, 176, 180, 172, 165, 168, 157, 153, 148, 143, 145, 138, 125, 118, 110, 92, 62, 33, 15, 8, 7, 10, 69, 64, 59, 57, 48, 69, 184, 175, 181, 144, 159, 166, 166, 173, 185, 190, 194, 202, 200, 195, 195, 195, 195, 193, 187, 178, 173, 179, 181, 171, 166, 161, 150, 147, 144, 145, 137, 126, 115, 83, 68, 56, 31, 20, 14, 9, 8, 11, 69, 61, 58, 49, 62, 91, 185, 179, 179, 172, 140, 154, 158, 159, 170, 180, 189, 201, 199, 198, 200, 195, 193, 192, 187, 182, 181, 184, 178, 168, 159, 154, 145, 141, 139, 131, 123, 104, 124, 40, 32, 28, 18, 18, 14, 11, 13, 13, 65, 61, 57, 73, 64, 112, 179, 179, 182, 177, 160, 129, 139, 143, 152, 166, 175, 190, 196, 196, 195, 188, 183, 180, 178, 181, 185, 183, 169, 163, 152, 141, 132, 128, 120, 106, 104, 128, 121, 10, 14, 17, 17, 17, 13, 13, 17, 19, 66, 63, 68, 71, 15, 186, 181, 176, 181, 181, 173, 154, 133, 122, 126, 143, 155, 169, 176, 181, 182, 180, 175, 176, 175, 173, 171, 165, 160, 147, 131, 119, 109, 108, 99, 97, 133, 142, 97, 11, 14, 14, 14, 12, 13, 16, 19, 25, 68, 73, 72, 44, 57, 188, 185, 189, 174, 187, 185, 176, 146, 139, 140, 135, 134, 141, 153, 162, 166, 167, 164, 164, 165, 159, 154, 144, 135, 121, 107, 100, 92, 98, 102, 109, 136, 140, 54, 15, 15, 15, 13, 11, 14, 18, 22, 26, 78, 71, 65, 35, 81, 188, 190, 183, 188, 178, 188, 189, 170, 147, 143, 148, 139, 128, 122, 128, 130, 133, 135, 135, 128, 125, 124, 113, 106, 100, 98, 99, 106, 102, 109, 130, 138, 134, 17, 18, 17, 17, 16, 14, 15, 22, 25, 29, 71, 69, 66, 14, 77, 189, 188, 185, 186, 187, 194, 188, 182, 167, 145, 140, 141, 139, 122, 105, 107, 101, 97, 96, 106, 99, 96, 94, 95, 96, 101, 111, 105, 101, 119, 140, 139, 126, 9, 20, 17, 17, 18, 16, 17, 23, 27, 32, 61, 61, 63, 11, 68, 187, 181, 182, 190, 196, 192, 189, 188, 180, 169, 151, 145, 143, 143, 137, 137, 133, 123, 119, 126, 124, 120, 111, 106, 104, 107, 106, 100, 118, 132, 140, 143, 101, 17, 22, 23, 23, 21, 21, 23, 26, 29, 34])

matrix([[0.]])

#### Utilizando Scikit Learn

In [74]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1000, solver='saga')
logreg.fit(x_trainG,y_trainG)
y_predG=logreg.predict(x_testG)
cnf_matrix = metrics.confusion_matrix(y_testG, y_predG)
cnf_matrix

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


array([[2601,  474],
       [ 472, 2380]], dtype=int64)

In [79]:
print("Accuracy para el género: ", metrics.accuracy_score(y_testG, y_predG))

Accuracy para el género:  0.8403914290534841


# Modelo para la Etnia

## Regresiones Logísticas

Etnia 0

In [10]:
start = time.time()
### Modelar 0

x0 = x_trainE
y0 = y_trainE
z0 = y_trainE

z0 = numpy.where(y0 != 0, 50, z0)
z0 = numpy.where(z0 == 0, 1, z0)
z0 = numpy.where(z0 == 50, 0, z0)

ceros = numpy.concatenate((x0[:,:], z0[:,:]), axis = 1)

P0 = ceros[ceros[ :, -1] == 1]
N0 = ceros[ceros[ :, -1] == 0]
numP0 = 3000
numN0 = 3000
numpy.random.shuffle( P0 )
numpy.random.shuffle( N0 )
data0 = numpy.concatenate( ( P0[ : numP0 , : ], N0[ : numN0 , : ] ), axis = 0 )
numpy.random.shuffle( data0 )

# Configure model
e0 = PUJ.Model.Logistic( )
e0.setParameters( [ 0 for i in range( data0.shape[ 1 ] ) ] )
#print( 'Initial model = ' + str( e0 ) )

# Configure cost
J0 = PUJ.Model.Logistic.Cost( e0, data0[ : , 0 : -1 ], data0[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple
##debugger = PUJ.Optimizer.Debug.PlotPolynomialCost( data[ : , 0 : -1 ], data[ : , -1 : ] )

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J0 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 20000 )
opt.setNumberOfDebugIterations( 500 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations :', opt.realIterations( ) )
print( '= Fitted model :', e0 )
print( '===========================================' )

y_real = data0[ : , -1 : ]
y_est = e0.threshold( data0[ : , 0 : -1 ] )

E0 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  E0[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1
# end for

print(E0)
accuracyE0 = (E0[0,0] + E0[1,1])/(E0[0,0] + E0[0,1] + E0[1,0] + E0[1,1])
print(accuracyE0)

#print(numpy.shape(e0))

end0 = time.time()

print("Time of execution of '0' model is :", end0-start)

0 0.6913338060261003 0.0018133745318449357
500 0.5868082547393726 9.394638006665623e-05
1000 0.5530062240150269 4.952945046721169e-05
1500 0.5332669047746144 3.161029978493435e-05
2000 0.5200301092322097 2.2249422017339704e-05
2500 0.510396701675867 1.673678073454976e-05
3000 0.5029707384842593 1.3216288562878376e-05
3500 0.4969969738309383 1.082695132353928e-05
4000 0.4920321257623867 9.125987608105124e-06
4500 0.4877992525702507 7.867484690449178e-06
5000 0.4841164917042229 6.906293041675227e-06
5500 0.48085944215968507 6.1523722684198034e-06
6000 0.4779400598389622 5.547506157665971e-06
6500 0.47529420567566005 5.052712682751537e-06
7000 0.4728739804403414 4.641094076829333e-06
7500 0.4706428318976701 4.293602236693506e-06
8000 0.4685723322245122 3.996437955178234e-06
8500 0.4666399967583413 3.7394021061842864e-06
9000 0.46482777172817313 3.5148207392921016e-06
9500 0.4631209632806044 3.316826773902548e-06
10000 0.46150746452471236 3.1408692865997345e-06
10500 0.45997718810415456 2.

Etnia 1

In [6]:
## Modelar 1

x1 = x_trainE
y1 = y_trainE
z1 = y_trainE

z1 = numpy.where(y1 != 1, 70, z1)
z1 = numpy.where(z1 == 1, 1, z1)
z1 = numpy.where(z1 == 70, 0, z1)

unos = numpy.concatenate((x1[:,:], z1[:,:]), axis = 1)

P1 = unos[unos[ :, -1] == 1]
N1 = unos[unos[ :, -1] == 0]
numP1 = 3000
numN1 = 3000
numpy.random.shuffle( P1 )
numpy.random.shuffle( N1 )
data1 = numpy.concatenate( ( P1[ : numP1 , : ], N1[ : numN1 , : ] ), axis = 0 )
numpy.random.shuffle( data1 )


# Configure model
e1 = PUJ.Model.Logistic( )
e1.setParameters( [ 0 for i in range( data1.shape[ 1 ] ) ] )

# Configure cost
J1 = PUJ.Model.Logistic.Cost( e1, data1[ : , 0 : -1 ], data1[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J1 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 20000 )
opt.setNumberOfDebugIterations( 500 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations :', opt.realIterations( ) )
print( '= Fitted model :', e1 )
print( '===========================================' )

y_real = data1[ : , -1 : ]
y_est = e1.threshold( data1[ : , 0 : -1 ] )
E1 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  E1[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1

accuracyE1 = (E1[0,0] + E1[1,1])/(E1[0,0] + E1[0,1] + E1[1,0] + E1[1,1])
print(accuracyE1)

#print(numpy.shape(e1))

end1 = time.time()

print("Time of execution of '1' model is :", end1-end0)

0 0.6857794509389847 0.0073677296189604835
500 0.5113076180234162 0.00014541914198729078
1000 0.4603439847612382 7.245044416959168e-05
1500 0.43200480384495177 4.455908584183321e-05
2000 0.41356306579544405 3.063860160967824e-05
2500 0.4004143278928993 2.2634200238180746e-05
3000 0.3904516342371843 1.7580682687923233e-05
3500 0.382567240093069 1.4170793330048337e-05
4000 0.3761200297881544 1.1752415525201343e-05
4500 0.37071180524261776 9.969401291332058e-06
5000 0.36608147039231154 8.613282982405934e-06
5500 0.36205030686462725 7.5551606607549004e-06
6000 0.3584916299163122 6.711706937057027e-06
6500 0.3553129938738072 6.027031991651111e-06
7000 0.352445265027556 5.4624663067825985e-06
7500 0.34983564749684076 4.990535257876072e-06
8000 0.3474430833338412 4.591265162778857e-06
8500 0.3452351314071654 4.249841701498003e-06
9000 0.34318579673770544 3.955081761153334e-06
9500 0.34127398773226497 3.698410285246645e-06
10000 0.3394823984013238 3.4731595263126636e-06
10500 0.337796684485207

Etnia 2

In [7]:
# Modelar Dos

x2 = x_trainE
y2 = y_trainE
z2 = y_trainE
z2 = numpy.where(y2 != 2, 0, z2)
z2 = numpy.where(z2 == 2, 1, z2)
dos = numpy.concatenate((x2[:,:], z2[:,:]), axis = 1)

P2 = dos[dos[ :, -1] == 1]
N2 = dos[dos[ :, -1] == 0]
numP2 = 3000
numN2 = 3000
numpy.random.shuffle( P2 )
numpy.random.shuffle( N2 )
data2 = numpy.concatenate( ( P2[ : numP2 , : ], N2[ : numN2 , : ] ), axis = 0 )
numpy.random.shuffle( data2 )

# Configure model
e2 = PUJ.Model.Logistic( )
e2.setParameters( [ 0 for i in range( data2.shape[ 1 ] ) ] )
#print( 'Initial model = ' + str( e2 ) )

# Configure cost
J2 = PUJ.Model.Logistic.Cost( e2, data2[ : , 0 : -1 ], data2[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple
##debugger = PUJ.Optimizer.Debug.PlotPolynomialCost( data[ : , 0 : -1 ], data[ : , -1 : ] )

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J2 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 20000 )
opt.setNumberOfDebugIterations( 500 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations       :', opt.realIterations( ) )
print( '= Fitted model     :', e2 )
print( '===========================================' )

y_real = data2[ : , -1 : ]
y_est = e2.threshold( data2[ : , 0 : -1 ] )
E2 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  E2[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1

accuracy2 = (E2[0,0] + E2[1,1])/(E2[0,0] + E2[0,1] + E2[1,0] + E2[1,1])
print(accuracy2)

#print(numpy.shape(e2))

end2 = time.time()

print("Time of execution of '2' model is :", end2-end1)

0 0.6915757593213021 0.0015714212366431113
500 0.5238440611136714 0.00013823117896238912
1000 0.4757498929845301 6.839400867653112e-05
1500 0.44878522738561694 4.284107014118632e-05
2000 0.4308891958575951 3.0033641419402457e-05
2500 0.4178921862025465 2.2569756308210565e-05
3000 0.4078842851949534 1.7796369956801783e-05
3500 0.3998500219770792 1.453944341239799e-05
4000 0.3931949239279798 1.2207009421616455e-05
4500 0.3875461460853611 1.0472185782817167e-05
5000 0.3826572575634184 9.141757885466895e-06
5500 0.37835847916746296 8.095264586449602e-06
6000 0.37452881959804485 7.254298002357107e-06
6500 0.3710795924404823 6.565974890127002e-06
7000 0.3679442167213032 5.993545421356661e-06
7500 0.3650716639412966 5.510811868503218e-06
8000 0.36242211166886396 5.09868257075885e-06
8500 0.3599639808597695 4.742973416016483e-06
9000 0.35767186799539247 4.432964821443619e-06
9500 0.35552507155115914 4.1604307949305586e-06
10000 0.3535065225695111 3.9189712095488716e-06
10500 0.3516019957380198

Etnia 3

In [8]:

## Modelar tres

x3 = x_trainE
y3 = y_trainE
z3 = y_trainE

z3 = numpy.where(y3 != 3, 0, z3)
z3 = numpy.where(z3 == 3, 1, z3)
# print(z)

tres = numpy.concatenate((x3[:,:], z3[:,:]), axis = 1)
# print(cincos)
P3 = tres[tres[ :, -1] == 1]
N3 = tres[tres[ :, -1] == 0]
# print('Positivos: ')
# print(P5)
# print('Negativos: ')
# print(N5)
numP3 = 3000
numN3 = 3000
numpy.random.shuffle( P3 )
numpy.random.shuffle( N3 )
data3 = numpy.concatenate( ( P3[ : numP3 , : ], N3[ : numN3 , : ] ), axis = 0 )
numpy.random.shuffle( data3 )
# print('Completo: ')
# print(data5)

# Configure model
e3 = PUJ.Model.Logistic( )
e3.setParameters( [ 0 for i in range( data3.shape[ 1 ] ) ] )
#print( 'Initial model = ' + str( e3 ) )

# Configure cost
J3 = PUJ.Model.Logistic.Cost( e3, data3[ : , 0 : -1 ], data3[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple
##debugger = PUJ.Optimizer.Debug.PlotPolynomialCost( data[ : , 0 : -1 ], data[ : , -1 : ] )

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J3 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 20000 )
opt.setNumberOfDebugIterations( 500 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations :', opt.realIterations( ) )
print( '= Fitted model :', e3 )
print( '===========================================' )

y_real = data3[ : , -1 : ]
y_est = e3.threshold( data3[ : , 0 : -1 ] )
E3 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  E3[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1

accuracy3 = (E3[0,0] + E3[1,1])/(E3[0,0] + E3[0,1] + E3[1,0] + E3[1,1])
print(accuracy3)

end3 = time.time()

print("Time of execution of '3' model is :", end3-end2)

0 0.6923053308428363 0.0008418497151089221
500 0.5758957505422877 9.971704690359129e-05
1000 0.5398350190320771 5.382734161363256e-05
1500 0.5178903908659165 3.6047680934125914e-05
2000 0.5025079645034902 2.6334588388410296e-05
2500 0.4909701241463467 2.0255833891369424e-05
3000 0.48193277893554304 1.6149344282956335e-05
3500 0.47462839167544346 1.3229360927902167e-05
4000 0.46857954533815943 1.1073327583932535e-05
4500 0.46347134149031516 9.43398558916897e-06
5000 0.45908681166578974 8.157597969904451e-06
5500 0.4552712583338911 7.144038798756469e-06
6000 0.45191127324927394 6.325638713278181e-06
6500 0.44892176870569206 5.655248104141641e-06
7000 0.4462376372969988 5.099160287180826e-06
7500 0.44380820886211503 4.632742991395045e-06
8000 0.44159346221964924 4.23764867341081e-06
8500 0.4395613731274942 3.8999805607176086e-06
9000 0.43768601827778186 3.6090561728041592e-06
9500 0.4359461944761589 3.3565549515945925e-06
10000 0.43432439634220216 3.135918988927955e-06
10500 0.43280604822

Etnia 4

In [9]:
### Modelar 4

x4 = x_trainE
y4 = y_trainE
z4 = y_trainE

z4 = numpy.where(y4 != 4, 0, z4)
z4 = numpy.where(z4 == 4, 1, z4)

cuatro = numpy.concatenate((x4[:,:], z4[:,:]), axis = 1)
P4 = cuatro[cuatro[ :, -1] == 1]
N4 = cuatro[cuatro[ :, -1] == 0]
numP4 = 3000
numN4 = 3000
numpy.random.shuffle( P4 )
numpy.random.shuffle( N4 )
data4 = numpy.concatenate( ( P4[ : numP4 , : ], N4[ : numN4 , : ] ), axis = 0 )
numpy.random.shuffle( data4 )

# Configure model
e4 = PUJ.Model.Logistic( )
e4.setParameters( [ 0 for i in range( data4.shape[ 1 ] ) ] )
#print( 'Initial model = ' + str( e4 ) )

# Configure cost
J4 = PUJ.Model.Logistic.Cost( e4, data4[ : , 0 : -1 ], data4[ : , -1 : ] )

# Debugger
debugger = PUJ.Optimizer.Debug.Simple
##debugger = PUJ.Optimizer.Debug.PlotPolynomialCost( data[ : , 0 : -1 ], data[ : , -1 : ] )

# Fit using an optimization algorithm
opt = PUJ.Optimizer.GradientDescent( J4 )
opt.setDebugFunction( debugger )
opt.setLearningRate( 1e-7 )
opt.setNumberOfIterations( 20000 )
opt.setNumberOfDebugIterations( 500 )
opt.Fit( )

# Show results
print( '===========================================' )
print( '= Iterations :', opt.realIterations( ) )
print( '= Fitted model :', e4 )
print( '===========================================' )

y_real = data4[ : , -1 : ]
y_est = e4.threshold( data4[ : , 0 : -1 ] )
E4 = numpy.zeros( ( 2, 2 ) )

for i in range( y_real.shape[ 0 ] ):
  E4[ int( y_real[ i, 0 ] ), int( y_est[ i, 0 ] ) ] += 1

accuracy4 = (E4[0,0] + E4[1,1])/(E4[0,0] + E4[0,1] + E4[1,0] + E4[1,1])
print(accuracy4)

#print(numpy.shape(e4))

end4 = time.time()

0 0.6115950638127972 0.08155211674514806
500 0.5774636591777366 3.538937698188693e-05
1000 0.5634534796912134 2.244938268392005e-05
1500 0.5539850768827701 1.606348299765248e-05
2000 0.5469671763806662 1.2310730456044361e-05
2500 0.5414598771324028 9.884317749841998e-06
3000 0.536960522540163 8.212709345922775e-06
3500 0.5331717632982561 7.006134611575732e-06
4000 0.5299052193888255 6.102952457731625e-06
4500 0.5270353286144551 5.406603611590732e-06
5000 0.5244750063956438 4.856323033597398e-06
5500 0.5221618691318121 4.41225838088144e-06
6000 0.5200500002876944 4.0473719323808055e-06
6500 0.5181048104948793 3.742789076333075e-06
7000 0.5162997120760732 3.4850068105773957e-06
7500 0.5146139014486865 3.264156288707021e-06
8000 0.5130308413278759 3.0728873404717305e-06
8500 0.5115371980074904 2.9056327989218644e-06
9000 0.510122082178659 2.7581117070241135e-06
9500 0.5087764967936452 2.6269867248185363e-06
10000 0.5074929290037314 2.509623391655502e-06
10500 0.5062650441638032 2.40391807

In [14]:
accgenero = round ( accuracy0 * 100 , 2 )
accetnia0 = round ( accuracyE0 * 100 , 2 )
accetnia1 = round ( accuracyE1 * 100 , 2 )
accetnia2 = round ( accuracy2 * 100 , 2 )
accetnia3 = round ( accuracy3 * 100 , 2 )
accetnia4 = round ( accuracy4 * 100 , 2 )

print(f'Accuracy para genero: {accgenero} %')
print(f'Accuracy para etnia 0: {accetnia0} %')
print(f'Accuracy para etnia 1: {accetnia1} %')
print(f'Accuracy para etnia 2: {accetnia2} %')
print(f'Accuracy para etnia 3: {accetnia3} %')
print(f'Accuracy para etnia 4: {accetnia4} %')

Accuracy para genero: 85.44 %
Accuracy para etnia 0: 80.5 %
Accuracy para etnia 1: 87.57 %
Accuracy para etnia 2: 87.26 %
Accuracy para etnia 3: 81.85 %
Accuracy para etnia 4: 75.95 %


## Red Neuronal

In [86]:
mlp = MLPClassifier(hidden_layer_sizes=(1000,200,20,1), activation='relu', solver='adam', max_iter=500)
mlp.fit(x_trainE,y_trainE)

predict_train = mlp.predict(x_trainE)
predict_test = mlp.predict(x_testE)

C:\Users\andre\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [87]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_trainE,predict_train))


[[7612    0    0    0    0]
 [3370    0    0    0    0]
 [2531    0    0    0    0]
 [2991    0    0    0    0]
 [1274    0    0    0    0]]


# Modelo para Edad

## Random Forest

In [75]:
# Inicializar claasificador
clf = RandomForestClassifier(n_estimators = 500)

# Entrenar el modelo con los datos de la etiqueda edad
clf.fit(x_trainA, y_trainA)

# Evaluar modelo con set de pruebas
y_pred = clf.predict(x_testA)

# using metrics module for accuracy calculation
print("Accuracy para la edad: ", metrics.accuracy_score(y_testA, y_pred))

Accuracy para la edad:  0.14560485911928464


### Guardar y cargar modelo

In [76]:
joblib.dump(clf, 'edad.joblib')

['edad.joblib']

In [77]:
modelocar2 = joblib.load('edad.joblib')

In [78]:
modelocar2.predict([[192, 192, 197, 207, 213, 213, 206, 197, 194, 200, 206, 210, 207, 207, 209, 214, 217, 215, 211, 215, 215, 216, 215, 212, 210, 207, 205, 211, 201, 201, 202, 203, 198, 194, 189, 184, 183, 181, 173, 164, 164, 164, 176, 187, 193, 196, 193, 182, 176, 185, 188, 208, 213, 207, 202, 195, 192, 198, 206, 210, 208, 207, 208, 214, 219, 215, 211, 213, 213, 216, 215, 212, 210, 207, 205, 211, 201, 201, 202, 202, 196, 193, 192, 191, 184, 178, 169, 163, 166, 171, 178, 186, 189, 185, 185, 173, 179, 169, 184, 201, 206, 202, 203, 193, 194, 204, 206, 208, 210, 209, 209, 211, 212, 210, 210, 211, 211, 214, 215, 214, 213, 208, 208, 209, 201, 199, 201, 200, 196, 194, 194, 195, 186, 173, 166, 164, 166, 169, 174, 188, 191, 192, 185, 179, 177, 160, 178, 191, 186, 190, 197, 194, 195, 204, 206, 207, 208, 210, 210, 211, 212, 212, 212, 215, 215, 215, 215, 213, 212, 211, 209, 206, 205, 201, 200, 199, 198, 196, 194, 188, 184, 181, 172, 161, 154, 163, 171, 182, 192, 187, 191, 186, 141, 163, 178, 187, 183, 179, 188, 188, 189, 202, 204, 208, 208, 213, 215, 214, 217, 215, 215, 217, 217, 214, 213, 212, 211, 211, 211, 207, 206, 201, 200, 198, 197, 194, 192, 187, 183, 183, 171, 157, 149, 152, 163, 174, 181, 193, 181, 190, 128, 174, 175, 175, 173, 169, 178, 181, 184, 197, 205, 205, 207, 207, 208, 207, 210, 210, 210, 211, 211, 213, 211, 211, 209, 211, 211, 207, 206, 204, 200, 195, 192, 191, 190, 186, 182, 181, 166, 151, 148, 157, 157, 166, 183, 179, 181, 187, 71, 164, 164, 166, 164, 166, 174, 177, 182, 200, 206, 209, 206, 208, 200, 200, 207, 214, 213, 210, 208, 208, 209, 211, 207, 206, 205, 207, 203, 201, 200, 193, 186, 190, 189, 183, 180, 183, 161, 148, 149, 153, 146, 149, 165, 172, 174, 170, 94, 158, 173, 170, 167, 167, 171, 178, 183, 197, 201, 205, 211, 212, 218, 209, 207, 204, 206, 206, 209, 202, 204, 206, 206, 207, 199, 199, 198, 191, 195, 194, 187, 188, 189, 177, 179, 174, 166, 148, 147, 142, 140, 144, 138, 173, 163, 165, 82, 155, 183, 174, 175, 172, 170, 176, 183, 185, 177, 174, 173, 175, 167, 175, 188, 198, 197, 192, 194, 192, 200, 197, 195, 195, 194, 180, 191, 194, 187, 180, 177, 181, 167, 162, 178, 170, 155, 155, 149, 147, 136, 141, 139, 159, 150, 168, 84, 159, 176, 169, 170, 182, 178, 177, 168, 138, 132, 134, 142, 104, 98, 82, 75, 122, 146, 169, 185, 189, 194, 189, 187, 178, 173, 161, 156, 123, 106, 94, 98, 111, 117, 134, 142, 159, 162, 150, 149, 148, 133, 137, 141, 146, 159, 153, 99, 167, 169, 164, 177, 177, 176, 176, 158, 140, 150, 141, 157, 150, 128, 93, 77, 78, 111, 121, 165, 178, 185, 190, 181, 165, 163, 141, 109, 76, 53, 45, 56, 79, 107, 125, 115, 115, 137, 147, 152, 140, 137, 140, 138, 142, 157, 161, 219, 183, 156, 166, 175, 177, 183, 182, 169, 149, 161, 165, 131, 102, 86, 82, 69, 65, 109, 123, 138, 172, 182, 192, 174, 152, 134, 98, 76, 57, 44, 57, 85, 119, 133, 135, 105, 102, 116, 146, 147, 136, 146, 136, 125, 133, 159, 169, 202, 186, 150, 154, 171, 180, 189, 181, 175, 169, 161, 62, 65, 68, 74, 56, 44, 70, 81, 108, 154, 164, 190, 194, 169, 116, 88, 74, 66, 85, 93, 90, 73, 73, 85, 107, 140, 121, 118, 132, 153, 153, 143, 132, 127, 125, 144, 160, 162, 174, 148, 147, 176, 172, 187, 180, 171, 135, 114, 108, 115, 100, 53, 46, 70, 121, 95, 112, 134, 161, 199, 201, 183, 122, 82, 75, 55, 77, 117, 77, 31, 56, 57, 49, 95, 153, 147, 156, 157, 160, 150, 136, 144, 122, 138, 155, 160, 171, 131, 146, 169, 181, 189, 182, 169, 151, 137, 141, 147, 142, 145, 138, 130, 103, 103, 137, 126, 161, 201, 205, 192, 131, 91, 93, 60, 74, 147, 69, 38, 62, 108, 91, 103, 140, 164, 174, 148, 163, 151, 144, 145, 128, 139, 147, 164, 165, 146, 167, 174, 181, 193, 197, 186, 197, 178, 170, 156, 185, 163, 141, 138, 124, 137, 138, 134, 175, 202, 204, 193, 141, 93, 114, 84, 96, 128, 144, 143, 152, 136, 116, 135, 152, 158, 165, 169, 166, 163, 144, 137, 125, 138, 151, 179, 168, 177, 167, 167, 177, 196, 202, 200, 210, 204, 200, 181, 180, 163, 149, 138, 134, 118, 132, 164, 182, 203, 206, 195, 161, 103, 83, 70, 75, 118, 166, 164, 169, 170, 167, 176, 177, 173, 173, 167, 161, 164, 153, 134, 121, 147, 153, 171, 182, 173, 145, 169, 178, 188, 201, 211, 199, 209, 209, 210, 198, 190, 174, 170, 160, 160, 170, 168, 177, 199, 205, 195, 154, 128, 115, 96, 101, 117, 140, 160, 174, 184, 186, 188, 186, 177, 168, 164, 161, 156, 158, 131, 118, 144, 150, 139, 194, 188, 174, 160, 178, 187, 194, 206, 205, 205, 207, 209, 203, 197, 193, 186, 183, 179, 171, 182, 173, 195, 211, 189, 142, 131, 127, 136, 142, 146, 168, 183, 193, 204, 207, 201, 191, 183, 170, 171, 158, 156, 145, 131, 124, 130, 152, 105, 201, 171, 192, 173, 172, 187, 191, 203, 212, 212, 210, 209, 206, 206, 209, 202, 200, 176, 174, 177, 170, 192, 204, 183, 146, 137, 135, 141, 157, 166, 178, 194, 205, 209, 201, 198, 189, 187, 172, 174, 156, 155, 130, 147, 148, 102, 128, 98, 201, 158, 194, 173, 167, 184, 189, 190, 196, 207, 206, 205, 202, 204, 206, 200, 186, 162, 175, 174, 178, 196, 203, 188, 164, 147, 141, 132, 146, 162, 178, 193, 199, 204, 195, 194, 188, 180, 166, 168, 152, 151, 130, 141, 154, 133, 150, 96, 200, 190, 174, 174, 166, 184, 186, 187, 193, 209, 211, 212, 208, 208, 206, 202, 180, 177, 177, 181, 178, 199, 203, 189, 170, 149, 144, 138, 152, 155, 166, 188, 194, 195, 190, 189, 188, 171, 161, 154, 152, 143, 134, 144, 162, 165, 145, 90, 204, 209, 173, 175, 163, 181, 189, 192, 191, 201, 211, 214, 208, 203, 198, 200, 190, 162, 178, 162, 174, 198, 198, 193, 176, 144, 139, 122, 139, 165, 173, 177, 183, 189, 197, 192, 185, 164, 155, 141, 152, 139, 136, 145, 169, 172, 131, 65, 194, 204, 197, 182, 154, 178, 195, 190, 193, 202, 207, 209, 207, 203, 198, 194, 177, 162, 180, 169, 187, 207, 204, 196, 191, 162, 146, 142, 121, 149, 165, 167, 176, 189, 195, 189, 174, 164, 151, 147, 142, 138, 141, 144, 163, 147, 158, 61, 170, 192, 186, 183, 154, 172, 187, 192, 193, 195, 205, 206, 204, 200, 196, 190, 162, 173, 185, 172, 194, 202, 197, 199, 195, 167, 144, 136, 125, 132, 137, 162, 173, 188, 190, 183, 170, 160, 151, 149, 140, 142, 142, 145, 159, 152, 188, 71, 127, 201, 194, 197, 163, 159, 186, 189, 190, 198, 204, 205, 204, 199, 194, 187, 158, 147, 120, 150, 180, 189, 191, 178, 175, 139, 100, 109, 126, 142, 129, 149, 169, 185, 183, 179, 162, 158, 151, 146, 141, 146, 139, 140, 165, 176, 183, 83, 70, 154, 200, 187, 162, 161, 174, 178, 187, 196, 207, 209, 205, 200, 194, 185, 171, 138, 121, 127, 148, 164, 165, 149, 132, 98, 80, 97, 129, 139, 140, 149, 170, 183, 177, 174, 156, 152, 152, 144, 145, 149, 139, 147, 155, 163, 172, 70, 67, 60, 112, 134, 165, 162, 163, 180, 185, 194, 200, 201, 201, 198, 192, 189, 171, 158, 147, 149, 144, 140, 131, 100, 93, 98, 118, 132, 136, 143, 154, 163, 179, 189, 182, 171, 154, 145, 152, 148, 141, 156, 140, 167, 167, 175, 158, 75, 58, 53, 41, 37, 137, 170, 167, 181, 183, 190, 194, 195, 196, 195, 196, 185, 177, 181, 174, 175, 154, 153, 151, 153, 145, 151, 159, 159, 155, 156, 154, 178, 185, 183, 183, 169, 152, 146, 153, 146, 149, 154, 136, 161, 167, 140, 57, 72, 68, 59, 54, 49, 115, 165, 174, 179, 185, 187, 189, 192, 190, 192, 194, 182, 190, 186, 183, 185, 163, 161, 157, 173, 161, 160, 159, 160, 157, 150, 150, 171, 177, 177, 173, 168, 158, 150, 151, 143, 155, 142, 43, 67, 40, 11, 29, 78, 77, 68, 59, 55, 106, 162, 173, 177, 187, 186, 187, 194, 187, 189, 183, 181, 186, 182, 180, 191, 180, 170, 168, 173, 172, 168, 162, 165, 157, 150, 153, 164, 177, 181, 168, 166, 159, 152, 151, 144, 154, 139, 6, 7, 14, 10, 18, 84, 83, 76, 65, 55, 97, 161, 170, 177, 188, 185, 186, 185, 180, 185, 166, 169, 179, 186, 185, 191, 181, 177, 176, 177, 174, 168, 162, 155, 136, 130, 142, 162, 165, 183, 170, 172, 154, 152, 149, 150, 153, 122, 4, 8, 13, 17, 15, 83, 82, 74, 66, 58, 88, 161, 170, 179, 187, 185, 187, 185, 189, 175, 150, 159, 169, 174, 177, 173, 171, 178, 184, 176, 145, 124, 125, 123, 117, 103, 101, 158, 172, 175, 172, 162, 155, 152, 149, 147, 153, 96, 8, 15, 22, 20, 18, 82, 78, 72, 65, 61, 80, 166, 162, 173, 185, 183, 186, 193, 186, 164, 125, 115, 94, 94, 95, 87, 63, 85, 102, 75, 47, 67, 68, 63, 63, 95, 99, 146, 181, 177, 169, 152, 156, 151, 146, 144, 152, 60, 20, 23, 22, 21, 21, 79, 75, 71, 67, 62, 66, 138, 158, 167, 183, 181, 186, 196, 182, 171, 173, 172, 173, 156, 171, 159, 164, 166, 161, 153, 149, 155, 152, 143, 139, 133, 141, 158, 190, 189, 170, 145, 160, 148, 143, 148, 149, 42, 30, 22, 16, 16, 16, 75, 72, 70, 67, 62, 78, 152, 166, 163, 178, 182, 176, 189, 189, 180, 185, 170, 164, 170, 172, 167, 172, 162, 150, 144, 154, 156, 152, 135, 123, 141, 153, 164, 180, 195, 162, 146, 147, 138, 143, 143, 136, 76, 24, 16, 9, 13, 12, 70, 69, 70, 67, 59, 65, 163, 154, 163, 165, 176, 172, 179, 189, 186, 189, 182, 174, 165, 168, 166, 154, 148, 142, 140, 147, 143, 137, 135, 135, 150, 163, 164, 172, 179, 148, 144, 144, 135, 142, 141, 136, 96, 41, 18, 13, 16, 13, 70, 67, 63, 63, 54, 67, 177, 150, 163, 151, 171, 169, 173, 185, 187, 191, 186, 177, 180, 165, 162, 156, 160, 161, 144, 155, 160, 157, 155, 158, 154, 164, 164, 160, 162, 133, 141, 144, 134, 135, 126, 159, 96, 47, 22, 15, 17, 15, 71, 66, 62, 60, 60, 57, 183, 166, 146, 173, 168, 166, 169, 180, 185, 187, 191, 190, 188, 187, 182, 178, 172, 171, 171, 168, 169, 171, 176, 166, 157, 167, 164, 156, 150, 126, 141, 143, 132, 129, 134, 130, 78, 37, 20, 12, 9, 11, 70, 65, 61, 58, 58, 61, 181, 175, 150, 161, 169, 166, 165, 177, 187, 190, 193, 196, 194, 192, 189, 186, 184, 180, 175, 172, 173, 176, 180, 172, 165, 168, 157, 153, 148, 143, 145, 138, 125, 118, 110, 92, 62, 33, 15, 8, 7, 10, 69, 64, 59, 57, 48, 69, 184, 175, 181, 144, 159, 166, 166, 173, 185, 190, 194, 202, 200, 195, 195, 195, 195, 193, 187, 178, 173, 179, 181, 171, 166, 161, 150, 147, 144, 145, 137, 126, 115, 83, 68, 56, 31, 20, 14, 9, 8, 11, 69, 61, 58, 49, 62, 91, 185, 179, 179, 172, 140, 154, 158, 159, 170, 180, 189, 201, 199, 198, 200, 195, 193, 192, 187, 182, 181, 184, 178, 168, 159, 154, 145, 141, 139, 131, 123, 104, 124, 40, 32, 28, 18, 18, 14, 11, 13, 13, 65, 61, 57, 73, 64, 112, 179, 179, 182, 177, 160, 129, 139, 143, 152, 166, 175, 190, 196, 196, 195, 188, 183, 180, 178, 181, 185, 183, 169, 163, 152, 141, 132, 128, 120, 106, 104, 128, 121, 10, 14, 17, 17, 17, 13, 13, 17, 19, 66, 63, 68, 71, 15, 186, 181, 176, 181, 181, 173, 154, 133, 122, 126, 143, 155, 169, 176, 181, 182, 180, 175, 176, 175, 173, 171, 165, 160, 147, 131, 119, 109, 108, 99, 97, 133, 142, 97, 11, 14, 14, 14, 12, 13, 16, 19, 25, 68, 73, 72, 44, 57, 188, 185, 189, 174, 187, 185, 176, 146, 139, 140, 135, 134, 141, 153, 162, 166, 167, 164, 164, 165, 159, 154, 144, 135, 121, 107, 100, 92, 98, 102, 109, 136, 140, 54, 15, 15, 15, 13, 11, 14, 18, 22, 26, 78, 71, 65, 35, 81, 188, 190, 183, 188, 178, 188, 189, 170, 147, 143, 148, 139, 128, 122, 128, 130, 133, 135, 135, 128, 125, 124, 113, 106, 100, 98, 99, 106, 102, 109, 130, 138, 134, 17, 18, 17, 17, 16, 14, 15, 22, 25, 29, 71, 69, 66, 14, 77, 189, 188, 185, 186, 187, 194, 188, 182, 167, 145, 140, 141, 139, 122, 105, 107, 101, 97, 96, 106, 99, 96, 94, 95, 96, 101, 111, 105, 101, 119, 140, 139, 126, 9, 20, 17, 17, 18, 16, 17, 23, 27, 32, 61, 61, 63, 11, 68, 187, 181, 182, 190, 196, 192, 189, 188, 180, 169, 151, 145, 143, 143, 137, 137, 133, 123, 119, 126, 124, 120, 111, 106, 104, 107, 106, 100, 118, 132, 140, 143, 101, 17, 22, 23, 23, 21, 21, 23, 26, 29, 34]])

array([52])

## Red neuronal